In [42]:
!pip install pandas pyreadstat huggingface_hub
!pip install openai tqdm
!pip install requests
!pip install selenium
!pip install nltk
!pip install pymupdf
!pip install fuzzywuzzy[speedup]
!pip install sentence-transformers
!pip install transformers datasets accelerate torch
!pip install -r requirements.txt
!pip install tiktoken


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ---------------------------------------- 893.9/893.9 kB 4.5 MB/s eta 0:00:00


In [52]:
import pandas as pd
import pyreadstat
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import nltk
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI
from dotenv import load_dotenv
import tiktoken
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import ast
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [53]:
#load datasets:
df_metaanalytic, meta_metaanalytic = pyreadstat.read_sav("C:/Users/janin/Downloads/ADS Thesis/Meta Analytic data.sav")
df_metaanalytic.columns = meta_metaanalytic.column_labels
df_systematicreview, meta_systematicreview = pyreadstat.read_sav("C:/Users/janin/Downloads/ADS Thesis/Systematic Review data.sav")
df_systematicreview.columns = meta_systematicreview.column_labels

#print(df_systematicreview.head())
#print(df_metaanalytic['Covidence paper ID'])
#print(df_systematicreview['Covidence study ID'])

#convert both IDs to strings for the same format
df_metaanalytic['Covidence paper ID'] = df_metaanalytic['Covidence paper ID'].astype(int).astype(str)
df_systematicreview['Covidence study ID'] = df_systematicreview['Covidence study ID'].astype(str)


#select relevant columns
df_meta_selected = df_metaanalytic[['Covidence paper ID', 'Hedges G (combined calculated and reported) at post', 'Outcome domains', 'Who or what was assessed?', 'Measurement type used to assess the outcome', 'Type of study']]
df_systematic_selected = df_systematicreview[['Covidence study ID', 'Name of the intervention', 'Title of the paper', 'Year of publication', 'Country intervention developed', 'Country study conducted', 'Mean age of participants (based on grade)', 'Type of study']]
#merge the two datasets based on their matching ID
df_merged = pd.merge(df_meta_selected, df_systematic_selected, left_on="Covidence paper ID", 
                     right_on="Covidence study ID", how="inner",  suffixes=('_meta', '_sysrev'))

#drop 1 ID to prevent duplicate columns
df_merged.drop(columns=['Covidence study ID'], inplace=True)

#create final dataframe with 148 unique interventions
final_df = df_merged.drop_duplicates(subset=['Name of the intervention'])
#number of unique interventions
final_df['Name of the intervention'].nunique()  #148 unique interventions
#save to csv with columns title, year of publication and hedge's G
final_df[['Name of the intervention', 'Title of the paper', 'Year of publication', 'Hedges G (combined calculated and reported) at post']].to_csv("full_dataframe.csv", index=False)


In [45]:

print(df_metaanalytic['Hedges G (combined calculated and reported) at post'].isna().sum())
print(len(df_metaanalytic['Covidence paper ID']))
print(final_df['Hedges G (combined calculated and reported) at post'].isna().sum())


182
2047
17


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import pandas as pd

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

title="The state of evidence for social and emotional learning: A contemporary meta-analysis of universal school-based SEL interventions."
driver.get("https://utrechtuniversity.on.worldcat.org/discovery")
try:
    #search title
    search_box = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "search-bar"))
    )
    search_box.clear()
    search_box.send_keys(title)
    search_box.send_keys(Keys.RETURN)
    print(f"Searching for: {title}")
    time.sleep(5)
     #find elements "View full text"
    elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'View full text')]")
    print(f"Found {len(elements)}'View full text' elements")

    if elements:
        print(f"clicking first 'View full text' element, {elements[0].tag_name}")
        driver.execute_script("arguments[0].click();", elements[0])
        time.sleep(5)
    else:
        print("No 'View full text' elements")

except Exception as e:
    print(f"REAL ERROR: {repr(e)}")

#WORKS UNTIL HERE

except Exception as e:
    print(f"Error after provider link: {repr(e)}")
#WORKS UNTIL HERE
try:
    #click provider link
    
    provider_link = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.LINK_TEXT, "View full text"))
    )
    print("Found provider 'View full text'")

    driver.execute_script("arguments[0].click();", provider_link)
    #pause so i can login
    time.sleep(5) 

    #switch to new tab
    if len(driver.window_handles) > 1:
        driver.switch_to.window(driver.window_handles[1])  #=the new tab
        print(f"switched to new tab {driver.current_url}")
    else:
        print("no new tab opened. still on WorldCat.")
        driver.quit()
        exit()
    print(f"Redirected to: {driver.current_url}")


    #manually login
    input("after login press enter here")
    time.sleep(5)
    full_page_text = driver.page_source
    print("Ready to extact full text")

except Exception as e:
    print(f"ERROR: {repr(e)}")

#EXTRACT FULL TEXT AFTER LOADING
import pandas as pd
time.sleep(5)
print(f"Current page URL before scraping: {driver.current_url}")

try:
    #extract paragraphs
    paragraphs = driver.find_elements(By.TAG_NAME, "p")
    full_text = "\n".join([para.text for para in paragraphs if para.text.strip() != ""])
    print(f"Extracted {len(full_text)} characters of full text.")

except Exception as e:
    print(f"Error extracting full text: {repr(e)}")
    full_text = ""

#save to csv

data = {
    "Title": [paper_title],
    "Year": [publication_year],
    "Full Text": [full_text]
}

df = pd.DataFrame(data)
df.to_csv("data.csv", index=False)
print("saved")


Searching for: The state of evidence for social and emotional learning: A contemporary meta-analysis of universal school-based SEL interventions.
Found 5'View full text' elements
clicking first 'View full text' element, p
Found provider 'View full text'
switched to new tab https://utrechtuniversity.idm.oclc.org/login?url=https://doi.org/10.1111/cdev.13968?nols=y
Redirected to: https://utrechtuniversity.idm.oclc.org/login?url=https://doi.org/10.1111/cdev.13968?nols=y
Ready to extact full text
Current page URL before scraping: https://srcd-onlinelibrary-wiley-com.utrechtuniversity.idm.oclc.org/doi/10.1111/cdev.13968
✅ Extracted 87562 characters from HTML.
Extracted title: The state of evidence for social and emotional learning: A contemporary meta-analysis of universal school-based SEL interventions
Extracted year: 2023
saved


In [ ]:
#TRY TO AUTOMATE FOR ALL PAPERS
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (StaleElementReferenceException,TimeoutException)
import time
import csv
import re
import os
import pandas as pd
#to download pdfs
import requests
import fitz
#load titles from full dataframe
titles_df = pd.read_csv("full_dataframe.csv")
#create folder to store .txt files
if not os.path.exists("papers"):
    os.makedirs("papers")

#clean titles for search input but keep the original title in csv
def clean_title_for_search(t):
    t = str(t)
    t = t.replace('\n', ' ').replace('\r', ' ')       #line breaks
    t = t.replace('’', "'").replace('‘', "'")         #apostrophes
    t = t.replace('“', '"').replace('”', '"')         #quotation
    t = t.replace('–', '-').replace('—', '-')         #dashes
    t = t.replace('…', '...')                         #dots
    t = re.sub(r'\s+', ' ', t)                        #spaces
    return t.strip()
#create list of clean titles
titles_df["Cleaned Title"] = titles_df["Title of the paper"].dropna().apply(clean_title_for_search)
titles = titles_df["Cleaned Title"].tolist()
#set up browser
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

#continue from where it left off if partially completed
try:
    results = pd.read_csv("partial_results.csv").to_dict("records")
    start_index = len(results)
    print(f"continue from paper {start_index + 1}")
except FileNotFoundError:
    results = []
    start_index = 0
#main scraping loop:
try:
    for idx, title in enumerate(titles[start_index:], start=start_index):
        print(f"\n search for {idx+1}/{len(titles)}: {title}")
        driver.get("https://utrechtuniversity.on.worldcat.org/discovery")
        #retry for search input
        for attempt in range(2):
            try:
                search_box = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.ID, "search-bar"))
                )
                search_box.clear()
                search_box.send_keys(title)
                search_box.send_keys(Keys.RETURN)
                break
            except StaleElementReferenceException:
                time.sleep(1)

        time.sleep(5)

        #try to find "View full text"
        found = False
        for attempt in range(2):
            try:
                WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH,
                    "//a[contains(text(), 'View full text') or "
                    "contains(text(), 'Bekijk volledige tekst') or "
                    "contains(text(), 'Volledige tekst bekijken') or "
                    "contains(@class, 'full-text') or "
                    "contains(@aria-label, 'full text')] | "
                    "//button[contains(text(), 'View full text') or "
                    "contains(text(), 'Bekijk volledige tekst') or "
                    "contains(text(), 'Volledige tekst bekijken') or "
                    "contains(@class, 'full-text') or "
                    "contains(@aria-label, 'full text')]"
                ))
                )
                elements = driver.find_elements(By.XPATH,
                    "//a[contains(text(), 'View full text') or "
                    "contains(text(), 'Bekijk volledige tekst') or "
                    "contains(text(), 'Volledige tekst bekijken') or "
                    "contains(@class, 'full-text') or "
                    "contains(@aria-label, 'full text')] | "
                    "//button[contains(text(), 'View full text') or "
                    "contains(text(), 'Bekijk volledige tekst') or "
                    "contains(text(), 'Volledige tekst bekijken') or "
                    "contains(@class, 'full-text') or "
                    "contains(@aria-label, 'full text')]"
                )
                for el in elements:
                    try:
                        if el.is_displayed():
                            driver.execute_script("arguments[0].click();", el)
                            time.sleep(5)
                            found = True
                            break
                    except Exception as e:
                        continue

            except Exception:
                print("can't find full text button")

            if not found:
                continue

        #wait for provider tab to open, then switch to it
        try:
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            driver.switch_to.window(driver.window_handles[-1])
            print(f" now on provider page: {driver.current_url}")
        except TimeoutException:
            print("tab couldn't open")
            continue

        #if not on UU wifi, manually login
        if "login" in driver.current_url.lower():
            input("press Enter to continue after loginning in manually")
            time.sleep(5)
        #extract title and year

        try:
            title_element = driver.find_element(By.TAG_NAME, "h1")
            paper_title = title_element.text
            print(f"Extracted title: {paper_title}")

        except Exception as e:
            print(f"Error extracting title: {repr(e)}")
            paper_title = ""

        try:
            page_text = driver.page_source
            import re
            match = re.search(r"(19|20)\d{2}", page_text)
            if match:
                publication_year = match.group(0)
            else:
                publication_year = "Unknown"
            print(f"Extracted year: {publication_year}")

        except Exception as e:
            print(f"Error extracting year: {repr(e)}")
            publication_year = "Unknown"

        #try to extract HTML full text
        try:
            article_divs = driver.find_elements(By.CSS_SELECTOR, "[class*='article-section'], [class*='fulltext']")
            full_text_blocks = [div.text for div in article_divs if div.text.strip()]
            full_text = "\n\n".join(full_text_blocks)
            print(f"Extracted {len(full_text)} characters")
        except:
            full_text = ""

        #detect if it is on summary page
        if len(full_text.strip()) < 500 or ("Abstract" in full_text and "Introduction" not in full_text):
            print("landed on summary page")
            full_text = ""

        #click download pdf if available
        if len(full_text.strip()) < 50:
            try:
                pdf_link = driver.find_element(By.PARTIAL_LINK_TEXT, "Download PDF")
                driver.execute_script("arguments[0].click();", pdf_link)
                time.sleep(5)
                driver.switch_to.window(driver.window_handles[-1])
            except:
                print("no pdf download link")
                full_text = ""

        #if html empty, fall back to pdf extraction
        if len(full_text.strip()) < 50 and ".pdf" in driver.current_url.lower():
            try:
                pdf_url = driver.current_url
                print(f"attempt to download PDF from {pdf_url}")
                response = requests.get(pdf_url)
                pdf_path = f"temp_paper_{idx}.pdf"

                with open(pdf_path, "wb") as f:
                    f.write(response.content)

                doc = fitz.open(pdf_path)
                full_text = "\n".join(page.get_text() for page in doc)
                doc.close()
                print(f"extracted {len(full_text)} characters from PDF")
            except Exception as e:
                print(f"couldn't extract from PDF {repr(e)}")
        #skip if not enough characters
        if len(full_text.strip()) < 200:
            print(f"skipped paper {idx+1}")
            txt_filename = "skipped paper"
            results.append({
                "Original Title": title,
                "Extracted Title": paper_title,
                "Year": publication_year,
                "Text File": txt_filename
            })
            df = pd.DataFrame(results)
            df.to_csv("partial_results.csv", index=False)
            continue

        #extract title
        try:
            paper_title = driver.find_element(By.TAG_NAME, "h1").text.strip()
        except:
            paper_title = title

        #extract publication year
        match = re.search(r"\b(19|20)\d{2}\b", driver.page_source)
        publication_year = match.group(0) if match else "Unknown"
        #clean file name so it makes sure to put correct name in
        def make_safe_filename(s):
            s = s.strip()
            s = s.replace('\n', ' ')                
            s = re.sub(r'[\\/*?:"<>|]', "", s)      
            s = re.sub(r'\s+', '_', s)             
            return s[:80]                         

        #save clean title for safe filename
        safe_title = make_safe_filename(title)
        txt_filename = f"paper_{idx+1:03d}_{safe_title}.txt"
        txt_path = os.path.join("papers", txt_filename)

        #per paper, save full text to .txt file
        if full_text.strip():
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(full_text)
        else:
            txt_filename = "None"  #so we know if missing

        #only save important info in csv
        results.append({
            "Original Title": title,
            "Extracted Title": paper_title,
            "Year": publication_year,
            "Text File": txt_filename
        })

        #save after each paper
        df = pd.DataFrame(results)
        df.to_csv("partial_results.csv", index=False)

        #restart browser after each 10 papers to prevent crashing / too much memory
        if (idx + 1) % 10 == 0:
            print("browser restarted")
            driver.quit()
            driver = webdriver.Chrome(options=options)

        #close tab + witch back safely
        try:
            current_handle = driver.current_window_handle
            for handle in driver.window_handles:
                if handle != current_handle:
                    driver.switch_to.window(handle)
                    driver.close()
            driver.switch_to.window(current_handle)
        except Exception as e:
            print(f" error: {e}")
except KeyboardInterrupt:
    print("\n Script paused")
    df = pd.DataFrame(results)
    df.to_csv("partial_results.csv", index=False)
    driver.quit()

#save all results
df = pd.DataFrame(results)
df.to_csv("full_dataframe_with_fulltexts.csv", index=False)

driver.quit()

continue from paper 6

 search for 6/148: Effects of the Cyberbullying Prevention Program Media Heroes (Medienhelden) on Traditional Bullying
can't find full text button
can't find full text button
tab couldn't open

 search for 7/148: Evaluation of a teacher-led, life-skills intervention for secondary school students in Uganda
 now on provider page: https://utrechtuniversity.idm.oclc.org/login?url=https://doi.org/10.1016/j.socscimed.2018.09.041?nols=y
Extracted title: Evaluation of a teacher-led, life-skills intervention for secondary school students in Uganda
Extracted year: 2018
Extracted 0 characters
landed on summary page
no pdf download link
skipped paper 7

 search for 8/148: Social and emotional training in Swedish classrooms for the promotion of mental health: results from an effectiveness study in Sweden
 now on provider page: https://go-gale-com.utrechtuniversity.idm.oclc.org/ps/i.do?ty=as&v=2.1&u=utrecht&it=DIourl&s=RELEVANCE&p=AONE&qt=TI%7E%22Immersive+Learning+and+Partici

In [ ]:
#COMPLETE DATASET MANUALLY
#load csv
df = pd.read_csv("full_dataframe.csv")
#create number paper column for matching
df.reset_index(inplace=True)
df.rename(columns={"index": "Paper number"}, inplace=True)
df["Paper number"] = df["Paper number"] + 1
#create full text column
df["Full text"] = ""
#find folder
paper_dir = "papers"
txt_files = [f for f in os.listdir(paper_dir) if f.endswith(".txt")]

inserted = []
unmatched = []

#loop through .txt files
for file in txt_files:
    match = re.match(r"paper_(\d+)_", file)
    if match:
        paper_num = int(match.group(1))
        #find the matching row
        match_row = df[df["Paper number"] == paper_num]
        if not match_row.empty:
            row_index = match_row.index[0]
            file_path = os.path.join(paper_dir, file)
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
            df.at[row_index, "Full text"] = content
            inserted.append((file, row_index + 2, paper_num))  #+2 for spreadsheet row
        else:
            unmatched.append(file)
    else:
        unmatched.append(file)
#check unmatched files
if unmatched:
    print("\n umatched .txt files:")
    for f in unmatched:
        print(f"- {f}")
#save to new csv
df.to_csv("full_dataframe_with_fulltext.csv", index=False)


In [ ]:
#GENERATE DESCRIPTIONS FROM INTERVENTIONS USING OPENAI API KEY
import pandas as pd
import time
from openai import OpenAI
from dotenv import load_dotenv
import os
project_key = "sk-proj" #deleted the key for github
#initalize OPENAI with key
client = OpenAI(api_key=project_key)
#load csv's
df_ft = pd.read_csv("full_dataframe_with_fulltext.csv")
df_ex = pd.read_csv("Manual descriptions.csv")

def count_tokens(text, model="gpt-3.5-turbo-16k-0613"):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

#few-shot prompts with 5 examples that i manually selected the descriptions
examples = df_ex.dropna(subset=["Intervention description"]).head(6)
few_shot_prompt = ""
#use manual descriptions.csv to train
for i, row in examples.iterrows():
    text = row["Full text"]
    few_shot_prompt += f"Example {i+1}:\n"
    few_shot_prompt += f"Text: {row['Full text']}\n"
    few_shot_prompt += f"Intervention: {row['Intervention description']}\n\n"
print("Few-shot prompt length (tokens):", count_tokens(few_shot_prompt))

#extract snippet around methods section, to avoid too many tokens
def methodology_snippet(text, max_chars=6000):
    keywords = ['methodology', 'methods', 'participants', 'procedure', 'design', 'intervention program', 'training program']
    for keyword in keywords:
        match = re.search(rf'\b{keyword}\b', text, re.IGNORECASE)
        if match:
            start_idx = match.start()
            return text[start_idx:start_idx + max_chars]
    #fallback is to start at 30% of paper
    fallback_start = int(len(text) * 0.3)
    return text[fallback_start:fallback_start + max_chars]
#generate descriptions
descriptions = []
#loop over each paper to generate descriptions
for i, row in df_ft.iterrows():
    snippet = methodology_snippet(row["Full text"])
    #build full prompt
    full_prompt = few_shot_prompt + f"Text: {snippet}\nIntervention:"
    print(f" full prompt size {count_tokens(full_prompt)} tokens")
    try:
        response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "user", "content": full_prompt}
        ],
        temperature =0.3,
        max_tokens = 150)

        description = response.choices[0].message.content.strip()
    except Exception as e:
        description = f"Error: {e}"

    descriptions.append(description)
    print(f"Processing {i+1}/{len(df_ft)}")
    time.sleep(1)
#add new column to dataframe
df_ft["Intervention description"] = descriptions
#save results
df_ft.to_csv("fulldata_withfulltext_withdescriptions.csv", index=False)



Few-shot prompt length (tokens): 4465
 full prompt size 5956 tokens
Processing 1/148
 full prompt size 5932 tokens
Processing 2/148
 full prompt size 5622 tokens
Processing 3/148
 full prompt size 5656 tokens
Processing 4/148
 full prompt size 5862 tokens
Processing 5/148
 full prompt size 5844 tokens
Processing 6/148
 full prompt size 4497 tokens
Processing 7/148
 full prompt size 5725 tokens
Processing 8/148
 full prompt size 5807 tokens
Processing 9/148
 full prompt size 5515 tokens
Processing 10/148
 full prompt size 5541 tokens
Processing 11/148
 full prompt size 5480 tokens
Processing 12/148
 full prompt size 6197 tokens
Processing 13/148
 full prompt size 5730 tokens
Processing 14/148
 full prompt size 5205 tokens
Processing 15/148
 full prompt size 6372 tokens
Processing 16/148
 full prompt size 5668 tokens
Processing 17/148
 full prompt size 5693 tokens
Processing 18/148
 full prompt size 5623 tokens
Processing 19/148
 full prompt size 5778 tokens
Processing 20/148
 full promp

In [13]:
#EMBEDDINGS from descriptions
from sentence_transformers import SentenceTransformer
import os
import csv
import pandas as pd

#load model (free)
model = SentenceTransformer("all-MiniLM-L6-v2")

#path
csv_path = "fulldata_withfulltext_withdescriptions.csv" 
df = pd.read_csv(csv_path) 
#embed generated intervention descriptions
text_column = "Intervention description"
df["Embedding"] = df[text_column].apply(lambda text: model.encode(str(text)).tolist())

#save 
df.to_csv("fulldatawithembeddings.csv", index=False)

In [ ]:
#supervised learning regression EMBEDDINGS
from sklearn.ensemble import GradientBoostingRegressor
#load csv
df = pd.read_csv("fulldatawithembeddings.csv")
#first filter out the NaN values in 'Hedges G (combined calculated and reported) at post'
df = df[df['Hedges G (combined calculated and reported) at post'].notna()]
print(len(df))
#seperate embeddings (features) from effect size (target)
#convert embedding strings to list
df["Embedding"] = df["Embedding"].apply(ast.literal_eval)
#convert embedding column lists to 2D numpy array
x_embed = np.vstack(df["Embedding"].values)
#Hedge's G as effect size for target data
y_embed = df['Hedges G (combined calculated and reported) at post'].values
#split data in train 80%, validation 10%, test 10%
#first split in 80% and 20% rest
x_train_embed, x_rest_embed, y_train_embed, y_rest_embed = train_test_split(x_embed, y_embed, test_size=0.2, random_state=42)
#split rest in 2 so we get 10% validation and 10% test
x_val_embed, x_test_embed, y_val_embed, y_test_embed = train_test_split(x_rest_embed, y_rest_embed, test_size=0.5, random_state=42)

#Linear regression
#train
model_linreg_embed = LinearRegression().fit(x_train_embed, y_train_embed)
#prediction with validation set
y_valpred_linreg_embed = model_linreg_embed.predict(x_val_embed)
#evaluate
rsq_linreg_embed = r2_score(y_val_embed, y_valpred_linreg_embed)
mse_linreg_embed =mean_squared_error(y_val_embed, y_valpred_linreg_embed)
rmse_linreg_embed =np.sqrt(mse_linreg_embed)
print(f"Linear regression evaluation embeddings only:")
print(f"R²: {rsq_linreg_embed}")
print(f"MSE: {mse_linreg_embed}")
print(f"RMSE:{rmse_linreg_embed}")

#Random forest
#train
model_rf_embed = RandomForestRegressor().fit(x_train_embed, y_train_embed)
#prediction with validation set
y_valpred_rf_embed = model_rf_embed.predict(x_val_embed)
#evaluate
rsq_rf_embed = r2_score(y_val_embed, y_valpred_rf_embed)
mse_rf_embed =mean_squared_error(y_val_embed, y_valpred_rf_embed)
rmse_rf_embed =np.sqrt(mse_rf_embed)
print(f"Random forest evaluation embeddings only:")
print(f"R²: {rsq_rf_embed}")
print(f"MSE: {mse_rf_embed}")
print(f"RMSE:{rmse_rf_embed}")

#Gradient boosting
#train
model_gb_embed = GradientBoostingRegressor().fit(x_train_embed, y_train_embed)
#prediction with validation set
y_valpred_gb_embed = model_gb_embed.predict(x_val_embed)

#evalute
rsq_gb_embed = r2_score(y_val_embed, y_valpred_gb_embed)
mse_gb_embed =mean_squared_error(y_val_embed, y_valpred_gb_embed)
rmse_gb_embed =np.sqrt(mse_gb_embed)
print(f"Gradient boosting evaluation embeddings only:")
print(f"R²: {rsq_gb_embed}")
print(f"MSE: {mse_gb_embed}")
print(f"RMSE:{rmse_gb_embed}")

131
Linear regression evaluation metrics:
R²: -1.3228002407252069
MSE: 0.5348103366311731
RMSE:0.7313072792138562
Random forest evaluation metrics:
R²: 0.3154801205458565
MSE: 0.15760645308324198
RMSE:0.3969967922833155
Gradient boosting evaluation metrics:
R²: 0.15717470241840592
MSE: 0.19405529292529725
RMSE:0.4405170744991586


In [ ]:
#hyperparameter tune random forest
#define paramater grid to try different values at once
parameter_grid_rf ={
    'n_estimators':[100,200], #number of decision trees in forest
    'max_features':["sqrt", "log2"], #sqrt selects sqrt of total features, log2 provides randomness
    'max_depth': [None, 10,20], #shallow tree underfits and deep tree overfits
    'min_samples_split': [2,5], #minnimum number of samples to split internal node into subnodes
    'min_samples_leaf': [1,2],  #minimum number of samples in a leaf node
    'bootstrap': [True, False]}
#grid search
grid_rf_embed = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=parameter_grid_rf, cv=5,  #5 fold CV
    scoring='neg_mean_squared_error', #returns MSE
    n_jobs=-1, #runs parallel to speed up
    verbose=1 #basic information printed
)
#fit grid on training
grid_rf_embed.fit(x_train_embed, y_train_embed)
best_rf_embed = grid_rf_embed.best_estimator_
#evaluate
y_valpred_rfhp_embed = best_rf_embed.predict(x_val_embed)
#evaluation metrics
rsq_rfhp_embed = r2_score(y_val_embed, y_valpred_rfhp_embed)
mse_rfhp_embed = mean_squared_error(y_val_embed, y_valpred_rfhp_embed)
rmse_rfhp_embed = np.sqrt(mse_rfhp_embed)
print("RF hyperparameter tuning embeddings only:")
print(f"R²: {rsq_rfhp_embed}")
print(f"MSE: {mse_rfhp_embed}")
print(f"RMSE: {rmse_rfhp_embed}")

#hyperparameter tune gradient boosting
from sklearn.ensemble import GradientBoostingRegressor
parameter_grid_gb ={
    'n_estimators':[100,200], #number of decision trees in forest
    'max_features':["sqrt", "log2"], #sqrt selects sqrt of total features, log2 provides randomness
    'max_depth': [None, 10,20], #shallow tree underfits and deep tree overfits
    'min_samples_split': [2,5], #minnimum number of samples to split internal node into subnodes
    'min_samples_leaf': [1,2],  #minimum number of samples in a leaf node 
}
grid_gb_embed = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42), param_grid=parameter_grid_gb, cv=5,  #5 fold CV
    scoring='neg_mean_squared_error', #returns MSE
    n_jobs=-1, #runs parallel to speed up
    verbose=1 #basic information printed
)
#fit grid on training
grid_gb_embed.fit(x_train_embed, y_train_embed)
best_gb_embed = grid_gb_embed.best_estimator_
#evaluate
y_valpred_gb_hp_embed = best_gb_embed.predict(x_val_embed)
#evaluation metrics
rsq_gb_hp = r2_score(y_val_embed, y_valpred_gb_hp_embed)
mse_gb_hp = mean_squared_error(y_val_embed, y_valpred_gb_hp_embed)
rmse_gb_hp = np.sqrt(mse_gb_hp)
print("GB hyperparameter tuning embeddings only:")
print(f"R²: {rsq_gb_hp}")
print(f"MSE: {mse_gb_hp}")
print(f"RMSE: {rmse_gb_hp}")


Fitting 5 folds for each of 96 candidates, totalling 480 fits
Final evaluation rf hyperparameter tuning:
R²: 0.16218207741966684
MSE: 0.19290237591456594
RMSE: 0.43920652990884135
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Final evaluation gb hyperparameter tuning:
R²: -0.016098015877477057
MSE: 0.23395026072153247
RMSE: 0.48368405051389946


In [ ]:
#predict with moderator variables only
#df_merged is systematic_review and meta_analytic combined and matched on paper ID
#remove rows with NaN for Hedges G
df1 = final_df[final_df['Hedges G (combined calculated and reported) at post'].notna()]

#only keep unique interventions
df_unique_int = df1.drop_duplicates(subset='Name of the intervention')
print(len(df_unique_int))
#save
df_unique_int.to_csv("all_mod.csv", index=False)
print(list(df_unique_int.columns))

#Hedges G as target variable
y = df_unique_int['Hedges G (combined calculated and reported) at post'].values
#moderator variables
moderators = ['Outcome domains', 'Who or what was assessed?', 
              'Measurement type used to assess the outcome', 
              'Type of study_meta', 'Country intervention developed', 
              'Country study conducted', 'Mean age of participants (based on grade)', 
              'Type of study_sysrev']

#sort out missing values
df_moderators = df_unique_int[moderators + ['Hedges G (combined calculated and reported) at post']].dropna()
#moderators as independent variables
x_mod = df_moderators[moderators].values
#effect sizes as target variable
y_mod = df_moderators['Hedges G (combined calculated and reported) at post'].values
#split up in 80% training, 10% validation, 10% test set 
x_train_mod, x_rest_mod, y_train_mod, y_rest_mod = train_test_split(x_mod, y_mod, test_size=0.2, random_state=42)
x_val_mod, x_test_mod, y_val_mod, y_test_mod = train_test_split(x_rest_mod, y_rest_mod, test_size=0.5, random_state=42)

model_linreg_mod = LinearRegression().fit(x_train_mod, y_train_mod)
#prediction with validation set
y_valpred_linreg_mod = model_linreg_mod.predict(x_val_mod)
#evalute
print("Linear Regression with all moderators:")
print("R²:", r2_score(y_val_mod, y_valpred_linreg_mod))
print("MSE:", mean_squared_error(y_val_mod, y_valpred_linreg_mod))
print("RMSE:", np.sqrt(mean_squared_error(y_val_mod, y_valpred_linreg_mod)))

#Random forest
parameter_grid_rf_mod = {
    'n_estimators': [100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}
grid_rf_mod = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=parameter_grid_rf_mod,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
#train
grid_rf_mod.fit(x_train_mod, y_train_mod)
best_rf_mod = grid_rf_mod.best_estimator_
#prediction with validation set
y_valpred_rf_mod = best_rf_mod.predict(x_val_mod)

#evaluate
print(f"Random forest hyperparameter tuning with all moderators:")
print("R²:", r2_score(y_val_mod, y_valpred_rf_mod))
print("MSE:", mean_squared_error(y_val_mod, y_valpred_rf_mod))
print("RMSE:", np.sqrt(mean_squared_error(y_val_mod, y_valpred_rf_mod)))

#Gradient boosting
parameter_grid_gb_mod = {
    'n_estimators': [100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
grid_gb_mod = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_grid=parameter_grid_gb_mod,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
#train
grid_gb_mod.fit(x_train_mod, y_train_mod)
best_gb_mod = grid_gb_mod.best_estimator_
#prediction with validation set
y_valpred_gb_mod = best_gb_mod.predict(x_val_mod)

#evalute
print(f"Gradient boosting hyperparameter tuning with all moderators:")
print("R²:", r2_score(y_val_mod, y_valpred_gb_mod))
print("MSE:", mean_squared_error(y_val_mod, y_valpred_gb_mod))
print("RMSE:", np.sqrt(mean_squared_error(y_val_mod, y_valpred_gb_mod)))


131
['Covidence paper ID', 'Hedges G (combined calculated and reported) at post', 'Outcome domains', 'Who or what was assessed?', 'Measurement type used to assess the outcome', 'Type of study_meta', 'Name of the intervention', 'Title of the paper', 'Year of publication', 'Country intervention developed', 'Country study conducted', 'Mean age of participants (based on grade)', 'Type of study_sysrev']
Linear Regression with moderators:
R²: -3.276993483999898
MSE: 0.06255983911865672
RMSE: 0.25011964960525734
Fitting 5 folds for each of 96 candidates, totalling 480 fits
Random forest with moderators:
R²: -0.9948082898446089
MSE: 0.02917817998837158
RMSE: 0.17081621699467409
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Gradient boosting with moderators:
R²: -1.7391573569584358
MSE: 0.04006581824664028
RMSE: 0.20016447798408257


In [ ]:
#combine embeddings + moderators
#load 
df_embed = pd.read_csv("fulldatawithembeddings.csv")
df_mod = pd.read_csv("all_mod.csv")
#merge on Title of the paper
df_merge = pd.merge(df_embed,df_mod, on='Title of the paper', how='inner')
#convert embeddings from strings to arrays
df_merge['Embedding'] = df_merge['Embedding'].apply(ast.literal_eval)

#moderators
moderators_full = ['Outcome domains', 'Who or what was assessed?', 
              'Measurement type used to assess the outcome', 
              'Type of study_meta', 'Country intervention developed', 
              'Country study conducted', 'Mean age of participants (based on grade)', 
              'Type of study_sysrev']
print(df_merge.columns.tolist())
#drop missing hedgds G
df_full = df_merge[moderators_full + ['Hedges G (combined calculated and reported) at post_x', 'Embedding']].dropna()
x_mod_full = df_full[moderators_full].values
x_embed_full = np.vstack(df_full['Embedding'].values)
y_full = df_full['Hedges G (combined calculated and reported) at post_x'].values
x_combi_full = np.hstack([x_mod_full, x_embed_full])
x_train_full, x_rest_full, y_train_full, y_rest_full = train_test_split(x_combi_full, y_full, test_size=0.2, random_state=42)
x_val_full, x_test_full, y_val_full, y_test_full = train_test_split(x_rest_full, y_rest_full, test_size=0.5, random_state=42)

lr_full = LinearRegression().fit(x_train_full, y_train_full)
y_pred_lr_full = lr_full.predict(x_val_full)
#linreg
print("Linear Regression (embeddings + 8 moderators):")
print("R²:", r2_score(y_val_full, y_pred_lr_full))
print("MSE:", mean_squared_error(y_val_full, y_pred_lr_full))
print("RMSE:", np.sqrt(mean_squared_error(y_val_full, y_pred_lr_full)))

#rf
rf_full = RandomForestRegressor(random_state=42).fit(x_train_full, y_train_full)
y_pred_rf_full = rf_full.predict(x_val_full)

print("Random Forest (embeddings + 8 moderators):")
print("R²:", r2_score(y_val_full, y_pred_rf_full))
print("MSE:", mean_squared_error(y_val_full, y_pred_rf_full))
print("RMSE:", np.sqrt(mean_squared_error(y_val_full, y_pred_rf_full)))

#gradient booster
gb_full = GradientBoostingRegressor(random_state=42).fit(x_train_full, y_train_full)
y_pred_gb_full = gb_full.predict(x_val_full)

print("Gradient Boosting (embeddings + 8 moderators):")
print("R²:", r2_score(y_val_full, y_pred_gb_full))
print("MSE:", mean_squared_error(y_val_full, y_pred_gb_full))
print("RMSE:", np.sqrt(mean_squared_error(y_val_full, y_pred_gb_full)))

#now only with embeddings and 2 moderators
moderators_2 =['Country study conducted', 'Mean age of participants (based on grade)']
#drop rows with NaN
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
df_2mod = df_merge[moderators_2 + ['Hedges G (combined calculated and reported) at post_x', 'Embedding']].dropna()
#features
x_mod_2 = df_2mod[moderators_2].values
x_embed_2 = np.vstack(df_2mod['Embedding'].values)
#combine
x_combi_2 = np.hstack([x_mod_2, x_embed_2])
#target
y_2 = df_2mod['Hedges G (combined calculated and reported) at post_x'].values

#train
x_train_2, x_rest_2, y_train_2, y_rest_2 = train_test_split(x_combi_2, y_2, test_size=0.2, random_state=42)
x_val_2, x_test_2, y_val_2, y_test_2 = train_test_split(x_rest_2, y_rest_2, test_size=0.5, random_state=42)

#LR
lr_2 = LinearRegression().fit(x_train_2, y_train_2)
y_pred_lr_2 = lr_2.predict(x_val_2)
print("Linear Regression (embeddings + 2 moderators):")
print("R²:", r2_score(y_val_2, y_pred_lr_2))
print("MSE:", mean_squared_error(y_val_2, y_pred_lr_2))
print("RMSE:", np.sqrt(mean_squared_error(y_val_2, y_pred_lr_2)))

#RF
rf_2 = RandomForestRegressor(random_state=42).fit(x_train_2, y_train_2)
y_pred_rf_2 = rf_2.predict(x_val_2)

print("Random Forest (embeddings + 2 moderators):")
print("R²:", r2_score(y_val_2, y_pred_rf_2))
print("MSE:", mean_squared_error(y_val_2, y_pred_rf_2))
print("RMSE:", np.sqrt(mean_squared_error(y_val_2, y_pred_rf_2)))

#RF hyperparameter tuning
param_grid_rf_2_hp = {
    'n_estimators': [100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}
rf_2_hp = RandomForestRegressor(random_state=42)

grid_search_rf_2_hp = GridSearchCV(
    estimator=rf_2_hp,
    param_grid=param_grid_rf_2_hp,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1)
grid_search_rf_2_hp.fit(x_train_2, y_train_2)
best_rf_2_hp = grid_search_rf_2_hp.best_estimator_
y_pred_rf_2_hp = best_rf_2_hp.predict(x_val_2)
print("Random Forest (embeddings + 2 moderators) hyperparameter tuning:")
print("R²:", r2_score(y_val_2, y_pred_rf_2_hp))
print("MSE:", mean_squared_error(y_val_2, y_pred_rf_2_hp))
print("RMSE:", np.sqrt(mean_squared_error(y_val_2, y_pred_rf_2_hp)))


#GB
gb_2 = GradientBoostingRegressor(random_state=42).fit(x_train_2, y_train_2)
y_pred_gb_2 = gb_2.predict(x_val_2)

print("Gradient Boosting (embeddings + 2 moderators):")
print("R²:", r2_score(y_val_2, y_pred_gb_2))
print("MSE:", mean_squared_error(y_val_2, y_pred_gb_2))
print("RMSE:", np.sqrt(mean_squared_error(y_val_2, y_pred_gb_2)))

#GB hyperparameter tuning
param_grid_gb_2_hp = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']}

gb_model_2_hp = GradientBoostingRegressor(random_state=42)
grid_search_gb_2_hp = GridSearchCV(
    estimator=gb_model_2_hp,
    param_grid=param_grid_gb_2_hp,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1)
grid_search_gb_2_hp.fit(x_train_2, y_train_2)
best_gb_2_hp = grid_search_gb_2_hp.best_estimator_
y_pred_gb_2_hp = best_gb_2_hp.predict(x_val_2)
print("Gradient Boosting (embeddings + 2 moderators) hyperparameter tuning:")
print("R²:", r2_score(y_val_2, y_pred_gb_2_hp))
print("MSE:", mean_squared_error(y_val_2, y_pred_gb_2_hp))
print("RMSE:", np.sqrt(mean_squared_error(y_val_2, y_pred_gb_2_hp)))

['Paper number', 'Name of the intervention_x', 'Title of the paper', 'Year of publication_x', 'Hedges G (combined calculated and reported) at post_x', 'Full text', 'Intervention description', 'Embedding', 'Covidence paper ID', 'Hedges G (combined calculated and reported) at post_y', 'Outcome domains', 'Who or what was assessed?', 'Measurement type used to assess the outcome', 'Type of study_meta', 'Name of the intervention_y', 'Year of publication_y', 'Country intervention developed', 'Country study conducted', 'Mean age of participants (based on grade)', 'Type of study_sysrev']
Linear Regression (embeddings + all moderators):
R²: -7.832668428546132
MSE: 0.1291959686039815
RMSE: 0.3594384072466123
Random Forest (embeddings + all moderators):
R²: -1.9804831248483072
MSE: 0.04359570466588668
RMSE: 0.20879584446508193
Gradient Boosting (embeddings + all moderators):
R²: -2.8015498142648605
MSE: 0.05560549616726388
RMSE: 0.23580817663360165
Linear Regression (embeddings + 2 moderators):
R²

In [ ]:
#get info from title
title = "Creating Supportive Contexts for Early Adolescents during the First Year of Middle School: Impact of a Developmentally Responsive Multi-Component Intervention"

columns = [
    'Country study conducted',
    'Mean age of participants (based on grade)',
    'Hedges G (combined calculated and reported) at post',
    'Year of publication'
]
info = final_df.loc[final_df['Title of the paper'] == title, columns]
print(info)


     Country study conducted  Mean age of participants (based on grade)  \
873                      1.0                                        6.0   

     Hedges G (combined calculated and reported) at post  Year of publication  
873                                          -0.102369                 2020.0  
